In [185]:
import warnings
warnings.filterwarnings('ignore')

from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
from os.path import join
import pandas as pd
import numpy as np
import pathlib

from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
import plotly.express as px
import dash

In [190]:
%%time
APP_PATH = r'C:\Users\rmartinez4\Box\Personal Git\Data-Visualization-DSE241\Final Project\dash_plotly_code'

df_data = pd.read_pickle(join(APP_PATH, join('data', 'data.pkl'))) 
df_coord = pd.read_pickle(join(APP_PATH, join('data', 'coordinates.pkl'))) 

Wall time: 2.34 s


In [191]:
%%time
print(df_data.shape)
df_data.head()

(11594073, 5)
Wall time: 996 µs


,State,City,Time,Dose_Rate,Gamma_Count
0,AK,ANCHORAGE,2008-06-04 15:49:00,NaN,1322.0
1,AK,ANCHORAGE,2008-06-04 16:49:00,NaN,1215.0
2,AK,ANCHORAGE,2008-06-04 17:50:00,NaN,1223.0
3,AK,ANCHORAGE,2008-06-04 18:50:00,NaN,1238.0
4,AK,ANCHORAGE,2008-06-04 19:50:00,NaN,1266.0


In [192]:
df_test = df_data[df_data.State=='CA']

In [193]:
df_test.City.unique()

array(['ANAHEIM', 'BAKERSFIELD', 'EUREKA', 'FRESNO', 'LOS ANGELES',
       'RIVERSIDE', 'SACRAMENTO', 'SAN BERNARDINO', 'SAN DIEGO',
       'SAN FRANCISCO', 'SAN JOSE'], dtype=object)

In [169]:
%%time
len(df_data)

Wall time: 0 ns


11594073

In [170]:
# df_coord.head()

In [171]:
%%time
test = set(df_data.State)

Wall time: 858 ms


In [172]:
%%time
test = df_data.State.unique()

Wall time: 319 ms


In [188]:
%%time
dff = df_data.merge(pd.DataFrame({'State':['CA','AK']}), how='inner')
print(dff.shape)
dff.head()

(1309660, 5)
Wall time: 866 ms


,State,City,Time,Dose_Rate,Gamma_Count
0,AK,ANCHORAGE,2008-06-04 15:49:00,NaN,1322.0
1,AK,ANCHORAGE,2008-06-04 16:49:00,NaN,1215.0
2,AK,ANCHORAGE,2008-06-04 17:50:00,NaN,1223.0
3,AK,ANCHORAGE,2008-06-04 18:50:00,NaN,1238.0
4,AK,ANCHORAGE,2008-06-04 19:50:00,NaN,1266.0


In [189]:
dff.City.unique()

array(['ANCHORAGE', 'FAIRBANKS', 'JUNEAU', 'ANAHEIM', 'BAKERSFIELD',
       'EUREKA', 'FRESNO', 'LOS ANGELES', 'RIVERSIDE', 'SACRAMENTO',
       'SAN BERNARDINO', 'SAN DIEGO', 'SAN FRANCISCO', 'SAN JOSE'],
      dtype=object)

In [174]:
%%time
dff = df_data.merge(pd.DataFrame({'State':['CA','AK']}), how='inner')

Wall time: 917 ms


In [175]:
print(dff.Time.min(), dff.Time.max())

2006-06-12 11:02:00 2020-12-31 15:59:00


In [176]:
%%time
dff = dff.set_index(['Time','State', 'City'])

Wall time: 541 ms


In [177]:
dff.head()

,,,Dose_Rate,Gamma_Count
Time,State,City,,
2008-06-04 15:49:00,AK,ANCHORAGE,NaN,1322.0
2008-06-04 16:49:00,AK,ANCHORAGE,NaN,1215.0
2008-06-04 17:50:00,AK,ANCHORAGE,NaN,1223.0
2008-06-04 18:50:00,AK,ANCHORAGE,NaN,1238.0
2008-06-04 19:50:00,AK,ANCHORAGE,NaN,1266.0


In [182]:
%%time
dff2 = dff.rolling(int(50000)).mean().reset_index()

Wall time: 235 ms


In [184]:
%%time
dff = dff.dropna(subset=['Gamma_Count'])

Wall time: 1.44 s


In [36]:
%%time 
end_date = df_data.Time.max()
end_date

Wall time: 45 ms


Timestamp('2020-12-31 19:12:00')

In [23]:
%%time 
end_date = max(df_data.Time)

Wall time: 1min 20s


In [33]:
%%time
df_data.sort_values('Time').iloc[-1].Time

Wall time: 4.77 s


Timestamp('2020-12-31 19:12:00')

In [24]:
%%time
start_date = end_date - relativedelta(years=1)

Wall time: 0 ns


In [19]:
print(start_date, end_date)

2019-12-31 19:12:00 2020-12-31 19:12:00


In [25]:
groupby_value='day'

In [89]:
%%time
dff = df_data.loc[(df_data.Time > start_date) & (df_data.Time <= end_date)]
dff.shape

Wall time: 373 ms


(1035372, 5)

In [66]:
%%time
dff[groupby_value] = [getattr(i, groupby_value) for i in dff.Time] 

Wall time: 7.99 s


In [67]:
%%time
cols = ['State', 'City', groupby_value]    
dff_1 = dff.groupby(cols, as_index=False)['Dose_Rate', 'Gamma_Count'].median()

Wall time: 612 ms


In [68]:
dff_1

,State,City,day,Dose_Rate,Gamma_Count
0,AK,ANCHORAGE,1,33.0,1641.0
1,AK,ANCHORAGE,2,33.0,1670.0
2,AK,ANCHORAGE,3,33.0,1646.0
3,AK,ANCHORAGE,4,32.0,1626.0
4,AK,ANCHORAGE,5,32.0,1647.0
...,...,...,...,...,...
4316,WY,CASPER,27,100.0,5502.0
4317,WY,CASPER,28,99.0,5420.0
4318,WY,CASPER,29,101.0,5522.0
4319,WY,CASPER,30,102.0,5451.5


In [90]:
print(dff.shape)
dff.head()

(1035372, 5)


,State,City,Time,Dose_Rate,Gamma_Count
78091,AK,ANCHORAGE,2019-12-31 20:05:00,38.0,1815.0
78092,AK,ANCHORAGE,2019-12-31 21:05:00,43.0,1982.0
78093,AK,ANCHORAGE,2019-12-31 22:05:00,43.0,2155.0
78094,AK,ANCHORAGE,2019-12-31 23:05:00,44.0,2503.0
78095,AK,ANCHORAGE,2020-01-01 00:06:00,48.0,2583.0


In [93]:
dff['groupy_time'] = getattr(dff.Time.dt, 'year')

In [95]:
print(dff.shape)
dff.head()

(1035372, 6)


,State,City,Time,Dose_Rate,Gamma_Count,groupy_time
78091,AK,ANCHORAGE,2019-12-31 20:05:00,38.0,1815.0,2019
78092,AK,ANCHORAGE,2019-12-31 21:05:00,43.0,1982.0,2019
78093,AK,ANCHORAGE,2019-12-31 22:05:00,43.0,2155.0,2019
78094,AK,ANCHORAGE,2019-12-31 23:05:00,44.0,2503.0,2019
78095,AK,ANCHORAGE,2020-01-01 00:06:00,48.0,2583.0,2020


In [78]:
%%time
cols = ['State', 'City', dff.Time.dt.month]    
dff_2 = dff.groupby(cols, as_index=False)['Dose_Rate', 'Gamma_Count'].median()

Wall time: 770 ms


In [79]:
dff_2

,State,City,Dose_Rate,Gamma_Count
0,AK,ANCHORAGE,33.0,1676.0
1,AK,ANCHORAGE,32.0,1562.0
2,AK,ANCHORAGE,30.0,1494.0
3,AK,ANCHORAGE,32.0,1601.0
4,AK,ANCHORAGE,31.0,1608.0
...,...,...,...,...
1612,WY,CASPER,101.0,5584.0
1613,WY,CASPER,100.0,5622.0
1614,WY,CASPER,101.0,5625.0
1615,WY,CASPER,103.0,5563.0


In [96]:
def merge_coordinates(data):
    dff = pd.merge(data, df_coord, on=['City', 'State'], how='inner')
    return dff

In [100]:
%%time
test_merge = merge_coordinates(dff)

test_merge.head()

Wall time: 751 ms


,State,City,Time,Dose_Rate,Gamma_Count,groupy_time,lat,lng,Elevation
0,AK,ANCHORAGE,2019-12-31 20:05:00,38.0,1815.0,2019,61.1508,-149.1091,236.45
1,AK,ANCHORAGE,2019-12-31 21:05:00,43.0,1982.0,2019,61.1508,-149.1091,236.45
2,AK,ANCHORAGE,2019-12-31 22:05:00,43.0,2155.0,2019,61.1508,-149.1091,236.45
3,AK,ANCHORAGE,2019-12-31 23:05:00,44.0,2503.0,2019,61.1508,-149.1091,236.45
4,AK,ANCHORAGE,2020-01-01 00:06:00,48.0,2583.0,2020,61.1508,-149.1091,236.45


In [101]:
%%time
test_merge = merge_coordinates(dff).to_json(date_format='iso', orient='split')

Wall time: 6.73 s
